# File prep for Hadoop wordcount

This notebook is used to prepare the data that will be given to the Hadoop wordcount program. The input will be a text file that contains the names of rated movies, which will also be given as JSON, a CSV-like file (taken from MySQL with Sqoop) and Flume tweets.

In [4]:
.libPaths()

[1] "/u02/anaconda3/lib/R/library"

In [6]:
# Load libraries
library('rjson')
library('dplyr')
library('readr')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




First of all, create the text file with the names of the movies.
For this, we take the movie name column from the movie ratings file and write them in a file line by line, so that the movie name appears several times (according to the many times it was rated) and wordcount produces a more interesting output

In [8]:
# Read the files to a data frame and join them
movies <- read.table("/u02/files/ArchivosPeliculas/u.item", header = TRUE, sep = "|")
rates <- read.table("/u02/files/ArchivosPeliculas/udata.data", header = TRUE, sep = "\t")
movies_df <- left_join(movies, rates, by = "itemid")
head(movies_df)

,itemid,title,rdate,vrdate,IMDbURL,unknown,Action,Adventure,Animation,Children,⋯,Musical,Mystery,Romance,SciFi,Thriller,War,Western,userid,rate,timestamp
,<int>,<chr>,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,⋯,0,0,0,0,0,0,0,308,4,887736532
2,1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,⋯,0,0,0,0,0,0,0,287,5,875334088
3,1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,⋯,0,0,0,0,0,0,0,148,4,877019411
4,1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,⋯,0,0,0,0,0,0,0,280,4,891700426
5,1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,⋯,0,0,0,0,0,0,0,66,3,883601324
6,1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,⋯,0,0,0,0,0,0,0,5,4,875635748


In [9]:
# Extract the column with the name
movies_names <- movies_df$title

# Write the names to a file line by line
write_lines(movies_names, "/u02/files/ArchivosPeliculas/movies_names.txt")

In [33]:
# Using the rjson package, from the movies' names column, create a json file with the next format:
# { "names_list" : [ ...
		# "Star Wars", "James Bond" ,
# ... ] }

# Create a list with the names
names_list <- list("names_list" = movies_names)

# Convert the list to json
names_json <- toJSON(names_list)

# Write the json to a file
write_lines(names_json, "/u02/files/ArchivosPeliculas/movies_names.json")

The imported files from MySQL follow a CSV like format, so it would also be suitable to transform the data to that format so that Sqoop has success when inserting it

In [16]:
# Write the movie names column to a csv file, with the next format:
# 1, Star Wars
# 2, James Bond
# ...

# Add the number of the movie to the names, which will follow from 1 to the number of movies
indexed_movies_names <- paste0(1:length(movies_names), ", ", movies_names)

# Write the names to a csv file
write_lines(indexed_movies_names, "/u02/files/ArchivosPeliculas/movies_names.csv")

In [35]:
# Check that all of the files were created as expected

# Read each file, but only the first lines
movies_text_file <- readLines("/u02/files/ArchivosPeliculas/movies_names.txt", n = 5)
movies_json_file <- fromJSON(file = "/u02/files/ArchivosPeliculas/movies_names.json") # This one is a single line
movies_csv_file <- readLines("/u02/files/ArchivosPeliculas/movies_names.csv", n = 5)

# Print them all
print(movies_text_file)
print(movies_json_file$names_list[1:5]) # Only print a couple of lines of the json file
print(movies_csv_file)

[1] "Toy Story (1995)" "Toy Story (1995)" "Toy Story (1995)" "Toy Story (1995)"
[5] "Toy Story (1995)"
[1] "Toy Story (1995)" "Toy Story (1995)" "Toy Story (1995)" "Toy Story (1995)"
[5] "Toy Story (1995)"
[1] "1, Toy Story (1995)" "2, Toy Story (1995)" "3, Toy Story (1995)"
[4] "4, Toy Story (1995)" "5, Toy Story (1995)"
